[Number of Unique Subjects Taught by Each Teacher ](https://leetcode.com/problems/number-of-unique-subjects-taught-by-each-teacher/description/?envType=study-plan-v2&envId=top-sql-50)

In [0]:
import pandas as pd

data = [[1, 2, 3], [1, 2, 4], [1, 3, 3], [2, 1, 1], [2, 2, 1], [2, 3, 1], [2, 4, 1]]
teacher = pd.DataFrame(data, columns=['teacher_id', 'subject_id', 'dept_id']).astype(
    {'teacher_id': 'Int64', 'subject_id': 'Int64', 'dept_id': 'Int64'})

In [0]:
result = teacher.groupby('teacher_id')['subject_id'].nunique().reset_index()
result.rename(columns={'subject_id':'cnt'}, inplace=True)
print(result)

   teacher_id  cnt
0           1    2
1           2    4


In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

teacher_df = spark.createDataFrame(teacher)
teacher_df.show()

+----------+----------+-------+
|teacher_id|subject_id|dept_id|
+----------+----------+-------+
|         1|         2|      3|
|         1|         2|      4|
|         1|         3|      3|
|         2|         1|      1|
|         2|         2|      1|
|         2|         3|      1|
|         2|         4|      1|
+----------+----------+-------+



In [0]:
import pyspark.sql.functions as F

teacher_df \
    .select('teacher_id', 'subject_id') \
    .groupBy('teacher_id') \
    .agg(F.count_distinct('subject_id').alias('cnt')) \
    .show()

+----------+---+
|teacher_id|cnt|
+----------+---+
|         1|  2|
|         2|  4|
+----------+---+



In [0]:
teacher_df.createOrReplaceTempView('teacher')

spark.sql('''
select 
    teacher_id,
    count(distinct subject_id) as cnt 
from teacher
group by teacher_id
''').show()

+----------+---+
|teacher_id|cnt|
+----------+---+
|         1|  2|
|         2|  4|
+----------+---+



[1141. User Activity for the Past 30 Days I](https://leetcode.com/problems/user-activity-for-the-past-30-days-i/description/?envType=study-plan-v2&envId=top-sql-50)

In [0]:
data = [[1, 1, '2019-07-20', 'open_session'], [1, 1, '2019-07-20', 'scroll_down'], [1, 1, '2019-07-20', 'end_session'],
        [2, 4, '2019-07-20', 'open_session'], [2, 4, '2019-07-21', 'send_message'], [2, 4, '2019-07-21', 'end_session'],
        [3, 2, '2019-07-21', 'open_session'], [3, 2, '2019-07-21', 'send_message'], [3, 2, '2019-07-21', 'end_session'],
        [4, 3, '2019-06-25', 'open_session'], [4, 3, '2019-06-25', 'end_session']]
activity = pd.DataFrame(data, columns=['user_id', 'session_id', 'activity_date', 'activity_type']).astype(
    {'user_id': 'Int64', 'session_id': 'Int64', 'activity_date': 'datetime64[ns]', 'activity_type': 'object'})

In [0]:
end_date = pd.to_datetime('2019-07-27')
start_date = end_date - pd.Timedelta(days=29)

# Filter the date range
filtered_df = activity[(activity['activity_date'] >= start_date) & 
                          (activity['activity_date'] <= end_date)]

# Group by activity_date and count distinct user_id
result = filtered_df.groupby('activity_date')['user_id'].nunique().reset_index()
result.rename(columns={'user_id': 'active_users'}, inplace=True)

print(result)

  activity_date  active_users
0    2019-07-20             2
1    2019-07-21             2


In [0]:

activity_df = spark.createDataFrame(activity)
activity_df.show()

+-------+----------+-------------------+-------------+
|user_id|session_id|      activity_date|activity_type|
+-------+----------+-------------------+-------------+
|      1|         1|2019-07-20 00:00:00| open_session|
|      1|         1|2019-07-20 00:00:00|  scroll_down|
|      1|         1|2019-07-20 00:00:00|  end_session|
|      2|         4|2019-07-20 00:00:00| open_session|
|      2|         4|2019-07-21 00:00:00| send_message|
|      2|         4|2019-07-21 00:00:00|  end_session|
|      3|         2|2019-07-21 00:00:00| open_session|
|      3|         2|2019-07-21 00:00:00| send_message|
|      3|         2|2019-07-21 00:00:00|  end_session|
|      4|         3|2019-06-25 00:00:00| open_session|
|      4|         3|2019-06-25 00:00:00|  end_session|
+-------+----------+-------------------+-------------+



In [0]:
activity_df.filter(
    (F.col('activity_date') <= '2019-07-27')
    &
    (F.col('activity_date') >= F.date_sub(F.lit('2019-07-27'), 29))) \
    .groupBy('activity_date').agg(F.count_distinct('user_id').alias('active_users')) \
    .show()

+-------------------+------------+
|      activity_date|active_users|
+-------------------+------------+
|2019-07-21 00:00:00|           2|
|2019-07-20 00:00:00|           2|
+-------------------+------------+



In [0]:
activity_df.createOrReplaceTempView('activity')

spark.sql('''
select activity_date as day, count(distinct user_id) as active_users
from activity
where activity_date<='2019-07-27' and activity_date>=date_sub('2019-07-27',  29)
group by activity_date;
''').show()

+-------------------+------------+
|                day|active_users|
+-------------------+------------+
|2019-07-21 00:00:00|           2|
|2019-07-20 00:00:00|           2|
+-------------------+------------+



[1070. Product Sales Analysis III](https://leetcode.com/problems/product-sales-analysis-iii/description/?envType=study-plan-v2&envId=top-sql-50)

In [0]:
data = [[1, 100, 2008, 10, 5000], [2, 100, 2009, 12, 5000], [7, 200, 2011, 15, 9000]]
sales = pd.DataFrame(data, columns=['sale_id', 'product_id', 'year', 'quantity', 'price']).astype(
    {'sale_id': 'Int64', 'product_id': 'Int64', 'year': 'Int64', 'quantity': 'Int64', 'price': 'Int64'})
data = [[100, 'Nokia'], [200, 'Apple'], [300, 'Samsung']]
product = pd.DataFrame(data, columns=['product_id', 'product_name']).astype(
    {'product_id': 'Int64', 'product_name': 'object'})

In [0]:
sales_df_sorted = sales.sort_values(by=['product_id', 'year'])

# Step 2: Get the first entry per product using groupby and first()
first_sales = sales_df_sorted.groupby('product_id', as_index=False).first()

# Step 3: Rename the year column to 'first_year'
first_sales.rename(columns={'year': 'first_year'}, inplace=True)

# View the final result
print(first_sales[['product_id', 'first_year', 'quantity', 'price']])

   product_id  first_year  quantity  price
0         100        2008        10   5000
1         200        2011        15   9000


In [0]:
spark = SparkSession.builder.getOrCreate()

sales_df = spark.createDataFrame(sales)
sales_df.show()

+-------+----------+----+--------+-----+
|sale_id|product_id|year|quantity|price|
+-------+----------+----+--------+-----+
|      1|       100|2008|      10| 5000|
|      2|       100|2009|      12| 5000|
|      7|       200|2011|      15| 9000|
+-------+----------+----+--------+-----+



In [0]:
product_df = spark.createDataFrame(product)
product_df.show()

+----------+------------+
|product_id|product_name|
+----------+------------+
|       100|       Nokia|
|       200|       Apple|
|       300|     Samsung|
+----------+------------+



In [0]:
from pyspark.sql import Window

In [0]:
sales_df \
    .withColumn('dn', F.dense_rank().over(Window.partitionBy('product_id').orderBy('year')))\
    .filter(F.col('dn')==1)\
    .select('product_id', F.col('year').alias('first_year'),'quantity','price')\
    .show()

+----------+----------+--------+-----+
|product_id|first_year|quantity|price|
+----------+----------+--------+-----+
|       100|      2008|      10| 5000|
|       200|      2011|      15| 9000|
+----------+----------+--------+-----+



In [0]:
sales_df.createOrReplaceTempView('sales')
product_df.createOrReplaceTempView('product')

spark.sql('''
select  
    product_id,
    year as first_year,
    quantity,
    price
from
(select 
    sales.*,
    dense_rank() over (partition by sales.product_id order by year) as dn
from sales
inner join product
on sales.product_id = product.product_id) t
where t.dn =1;
''').show()

+----------+----------+--------+-----+
|product_id|first_year|quantity|price|
+----------+----------+--------+-----+
|       100|      2008|      10| 5000|
|       200|      2011|      15| 9000|
+----------+----------+--------+-----+



[596. Classes More Than 5 Students](https://leetcode.com/problems/classes-more-than-5-students/description/?envType=study-plan-v2&envId=top-sql-50)

In [0]:
data = [['A', 'Math'], ['B', 'English'], ['C', 'Math'], ['D', 'Biology'], ['E', 'Math'], ['F', 'Computer'],
        ['G', 'Math'], ['H', 'Math'], ['I', 'Math']]
courses = pd.DataFrame(data, columns=['student', 'class']).astype({'student': 'object', 'class': 'object'})

In [0]:
class_count = courses.groupby('class').size().reset_index(name='class_count')
result = class_count[class_count['class_count'] >=5][['class']]
print(result)

  class
3  Math


In [0]:
courses_spark = spark.createDataFrame(courses)
courses_spark.show()

+-------+--------+
|student|   class|
+-------+--------+
|      A|    Math|
|      B| English|
|      C|    Math|
|      D| Biology|
|      E|    Math|
|      F|Computer|
|      G|    Math|
|      H|    Math|
|      I|    Math|
+-------+--------+



In [0]:
courses_spark.groupBy('class')\
             .agg(F.count('class').alias('class_count'))\
             .filter(F.col('class_count')>=5)\
             .select('class').show()
# 1 job = 1 action
# 1 wide transformation , 1 narrow transformation
# stages = 3

+-----+
|class|
+-----+
| Math|
+-----+



In [0]:
courses_spark.createTempView('courses')

spark.sql('''
          select class 
from courses
group by class
having count(class)>=5;
          ''').show()

+-----+
|class|
+-----+
| Math|
+-----+



[1729. Find Followers Count](https://leetcode.com/problems/find-followers-count/description/?envType=study-plan-v2&envId=top-sql-50)

In [0]:
data = [['0', '1'], ['1', '0'], ['2', '0'], ['2', '1']]
followers = pd.DataFrame(data, columns=['user_id', 'follower_id']).astype({'user_id': 'Int64', 'follower_id': 'Int64'})

In [0]:
grouped = followers.groupby('user_id')
followers_count = grouped['user_id'].count()
followers_count_df = followers_count.reset_index(name='followers_count')
followers_count_df = followers_count_df.sort_values('user_id')
print(followers_count_df)

   user_id  followers_count
0        0                1
1        1                1
2        2                2


In [0]:
followers_df = spark.createDataFrame(followers)
followers_df.show()

+-------+-----------+
|user_id|follower_id|
+-------+-----------+
|      0|          1|
|      1|          0|
|      2|          0|
|      2|          1|
+-------+-----------+



In [0]:
followers_df\
    .groupBy('user_id')\
    .agg(F.count('user_id').alias('followers_count'))\
    .orderBy('user_id')\
    .show()

# 1 action 1 job
# 2 wide transformation, 1 narrow transformation 
# 4 stages 

+-------+---------------+
|user_id|followers_count|
+-------+---------------+
|      0|              1|
|      1|              1|
|      2|              2|
+-------+---------------+



In [0]:
followers_df.createOrReplaceTempView('followers')

spark.sql('''
select 
    user_id,
    count(user_id) as followers_count
from followers
group by user_id
order by user_id;
''').show()

+-------+---------------+
|user_id|followers_count|
+-------+---------------+
|      0|              1|
|      1|              1|
|      2|              2|
+-------+---------------+



[619. Biggest Single Number](https://leetcode.com/problems/biggest-single-number/description/?envType=study-plan-v2&envId=top-sql-50)

In [0]:
data = [[8], [8], [3], [3], [1], [4], [5], [6]]  # first example
#data = [[8], [8], [7], [7], [3], [3], [3]] # second example
my_numbers = pd.DataFrame(data, columns=['num']).astype({'num': 'Int64'})

In [0]:
count_df = my_numbers.groupby('num').size().reset_index(name='count')

# Step 2: Filter to keep only numbers with count == 1
filtered_df = count_df[count_df['count'] == 1]

# Step 3: Find the maximum number among the filtered ones
result = filtered_df['num'].max()

# Step 4: Wrap in a DataFrame if you want the same structure
result_df = pd.DataFrame({'num': [result]})
print(result_df)

   num
0    6


In [0]:
my_numbers_df = spark.createDataFrame(my_numbers)
my_numbers_df.show()

+---+
|num|
+---+
|  8|
|  8|
|  3|
|  3|
|  1|
|  4|
|  5|
|  6|
+---+



In [0]:
my_numbers_df \
    .groupBy('num') \
    .agg(F.count('num').alias('count')) \
    .filter(F.col('count') == 1) \
    .agg(F.max('num').alias('num')) \
    .show()

+---+
|num|
+---+
|  6|
+---+



In [0]:
my_numbers_df.createOrReplaceTempView('MyNumbers')


spark.sql('''
select max(t.num) as num
from (
    select num
    from mynumbers
    group by num
    having count(num)=1
) t;
''').show()

+---+
|num|
+---+
|  6|
+---+



[1045. Customers Who Bought All Products](https://leetcode.com/problems/customers-who-bought-all-products/description/?envType=study-plan-v2&envId=top-sql-50)

In [0]:
data = [[1, 5], [2, 6], [3, 5], [3, 6], [1, 6]]
customer = pd.DataFrame(data, columns=['customer_id', 'product_key']).astype(
    {'customer_id': 'Int64', 'product_key': 'Int64'})
data = [[5], [6]]
product = pd.DataFrame(data, columns=['product_key']).astype({'product_key': 'Int64'})

In [0]:
total_product = product['product_key'].nunique()

customer_product_count = customer.groupby('customer_id')['product_key'].nunique().reset_index()
result_df = customer_product_count[customer_product_count['product_key'] == total_product][['customer_id']]

print(result_df)

   customer_id
0            1
2            3


In [0]:
customer_df = spark.createDataFrame(customer)
product_df = spark.createDataFrame(product)

In [0]:
total_product = product_df.select(F.countDistinct('product_key').alias('total')).collect()[0]['total']
customer_product_count = customer_df.groupBy('customer_id')\
                            .agg(F.countDistinct('product_key').alias('product_brought'))
result_df = customer_product_count.filter(F.col("product_brought") == total_product).select("customer_id").show()


+-----------+
|customer_id|
+-----------+
|          1|
|          3|
+-----------+



In [0]:
customer_df.createOrReplaceTempView('customer')
product_df.createOrReplaceTempView('product')

spark.sql(
    '''
    WITH product_total as (
        SELECT count(DISTINCT product_key) as total_products
        FROM product
    )
    
    SELECT customer_id
    FROM (
        SELECT customer_id, COUNT(DISTINCT product_key) AS products_bought
        FROM customer
        GROUP BY customer_id
    ) customer_summary
    CROSS JOIN product_total
    WHERE customer_summary.products_bought = product_total.total_products
    ''').show()

+-----------+
|customer_id|
+-----------+
|          1|
|          3|
+-----------+

